## 🧠 Importation des bibliothèques pour la modélisation

Dans cette cellule, nous importons toutes les bibliothèques nécessaires pour tester différents modèles d’apprentissage automatique sur nos datasets HOG.

### Modèles importés :

- **Arbres de décision et ensembles** :
  - `DecisionTreeClassifier` → arbre de décision simple  
  - `RandomForestClassifier` → forêt aléatoire  
  - `ExtraTreesClassifier` → forêt extrêmement aléatoire  
  - `GradientBoostingClassifier` → gradient boosting classique  
  - `HistGradientBoostingClassifier` → version optimisée pour grands datasets  

- **SVM et régression** :
  - `SVC` → support vector classifier  
  - `LogisticRegression` → régression logistique  

- **Autres modèles** :
  - `KNeighborsClassifier` → k-plus proches voisins  
  - `MLPClassifier` → perceptron multi-couches (réseau de neurones simple)  
  - `XGBClassifier` → XGBoost, boosting basé sur les arbres  

### Utilitaires :

- `pandas` → manipulation de DataFrame pour stocker et analyser les résultats  
- `joblib` → sauvegarde et chargement des splits et modèles  
- `time` → mesurer le temps d’entraînement  
- `tqdm` → barre de progression pour les boucles

> ⚡ Objectif : disposer de tous les modèles et outils nécessaires pour évaluer leurs performances sur nos datasets HOG.


In [6]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,HistGradientBoostingClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import pandas as pd
import joblib
import time
from tqdm import tqdm

## 📂 Chargement des splits prétraités

Dans cette cellule, nous chargeons les fichiers `.pkl` sauvegardés précédemment pour chaque taille d’image.  

Chaque fichier contient :  
- `X_train` : vecteurs de caractéristiques HOG normalisés pour l’entraînement  
- `X_test`  : vecteurs de caractéristiques HOG normalisés pour le test  
- `y_train` : labels d’entraînement  
- `y_test`  : labels de test  

Ces splits sont stockés dans un dictionnaire `split` indexé par la taille de l’image, ce qui permet de **travailler facilement sur différentes résolutions** :


In [2]:
split={}
tailles = [64,128,176,224,299]
for t in tailles:
    filename=f"splits_{t}.pkl"
    split[t]=joblib.load(filename)

## 🧩 Définition des modèles à tester

Dans cette cellule, nous préparons tous les modèles que nous allons entraîner et évaluer sur nos vecteurs HOG.  
L’objectif est de comparer les performances et de voir quels modèles sont les plus adaptés à notre dataset.

### Modèles classiques
Ces modèles n’utilisent pas d’arbres et sont adaptés pour des vecteurs de caractéristiques denses :  
- **SVC** : Support Vector Classifier  
- **KNN** : k-plus proches voisins  
- **MLP** : réseau de neurones simple avec deux couches cachées  
- **Logistic Regression** : régression logistique classique  

### Modèles basés sur les arbres
Ces modèles utilisent des arbres de décision simples ou des ensembles d’arbres pour améliorer les performances :  
- **DecisionTree** : arbre de décision simple  
- **RandomForest** : forêt aléatoire, réduction de la variance grâce au bagging  
- **ExtraTrees** : forêt extrêmement aléatoire  
- **GradientBoosting** : gradient boosting classique, réduction du biais  
- **XGBoost** : version optimisée de gradient boosting, adaptée aux grands datasets  

> 💡 Remarque : Cette distinction entre modèles classiques et arbres permet de comparer facilement **différents types de modèles** et de sélectionner ceux qui offrent le meilleur compromis entre biais, variance et performances.


In [4]:
# === Définition des modèles classiques  ===
model_classiques={
     "SVC": SVC(),
     "KNN": KNeighborsClassifier(),
     "MLP": MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=300, random_state=42),
     "LR": LogisticRegression(max_iter=10000, solver='lbfgs')}

# === Définition des modèles arbres  ===
model_arbres = {
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(n_jobs=-1, random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_jobs=-1, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "XGB": XGBClassifier(n_jobs=-1,  eval_metric='mlogloss')
}


## 🏃‍♂️ Expérimentation et évaluation des modèles

Cette cellule effectue les tests de performance pour tous les modèles définis précédemment, sur différentes tailles d’images et différents sous-ensembles d’échantillons.  

### Étapes principales

1. **Définition des tailles et échantillons**  
   - Tailles des images : 64, 128, 176, 224, 299  
   - Nombre d’images pour l’entraînement : 1664, 2664, 4664, 6664, 8664  

2. **Sélection des modèles**  
   - `Classique` : SVC, KNN, MLP, LogisticRegression  
   - `Arbre` : DecisionTree, RandomForest, ExtraTrees, GradientBoosting, XGBoost  

3. **Boucle d’expérimentation**  
   - Pour chaque type de modèle (`Classique` ou `Arbre`)  
   - Pour chaque taille d’échantillon  
   - Pour chaque taille d’image  

4. **Préparation des données**  
   - Chargement des splits train/test correspondants  
   - Test **sur toutes les tailles d’échantillons** pour observer si la quantité de données impacte les scores ou non  

5. **Entraînement et évaluation**  
   - Entraînement de chaque modèle sur le sous-échantillon correspondant  
   - Calcul du **score sur l’ensemble d’entraînement** (`score_train`)  
   - Calcul du **score sur l’ensemble de test** (`score_test`)  
   - Mesure du **temps d’entraînement** pour chaque modèle  
   - ⚠️ Remarque : certaines étapes peuvent prendre beaucoup de temps selon le modèle et la taille des images. Des redémarrages ou réinitialisations ont parfois été nécessaires pour éviter que le notebook ne reste bloqué.  

6. **Stockage des résultats**  
   - Tous les résultats sont ajoutés à une liste `resultat`  
   - Sauvegarde intermédiaire en CSV après chaque taille d’échantillon pour éviter toute perte  

> ⚡ Objectif : obtenir un **tableau complet des performances** pour chaque modèle, chaque taille d’image et chaque nombre d’échantillons, afin de comparer efficacement **biais, variance et temps d’entraînement**, et observer l’influence du nombre d’images sur les scores.


In [5]:
resultat=[]

# === Listes des tailles et échantillons ===
tailles = [64,128,176,224,299]
sample=[1664,2664,4664,6664,8664]

models = {
    "Classique": model_classiques,
    "Arbre": model_arbres
}

for type_modele, dico in models.items():
    
    print(f"\n==================== Tests des modèles {type_modele.upper()} ====================\n")
    
    # === Boucles d’expérimentation ===
    for s in sample:
        print(f"\n=== Traitement pour {s} images ===")
           
        for taille in tailles:
            print(f"\n=== Traitement pour la Taille {taille}x{taille} ===")
            X_train_scaled=split[taille]["X_train"]
            X_test_scaled=split[taille]["X_test"]
            y_train=split[taille]["y_train"]
            y_test=split[taille]["y_test"]
        
            # ✅ Réduction du jeu d'entraînement si nécessaire
            n = min(s, len(X_train_scaled))
            X_train_sub = X_train_scaled[:n]
            y_train_sub = y_train[:n]
        
            # === Entraînement et mesure de performance ===
            for clf_name,clf in dico.items():
                print(f"\n=== Traitement du model {clf_name} ===")
                start = time.time()
                clf.fit(X_train_sub,y_train_sub)
                duration = time.time() - start
        
                score_train=clf.score(X_train_sub,y_train_sub)
                score_test=clf.score(X_test_scaled,y_test)
                
                resultat.append({"Type": type_modele,"sample":n,"taille":taille,"clf":clf_name,"score_train":round(score_train,4),"score_test":round(score_test,4),"Temps(s)":round(duration,2)})
    
        # sauvegarde de sécurité après chaque taille d’échantillon
        pd.DataFrame(resultat).to_csv(f"resultats_modeles_{type_modele}_{s}.csv", index=False)
        print(f"\n Sauvegarde intermédiaire : resultats_modeles_{type_modele}_{s}.csv")




==================== Tests des modèles ARBRE ====================


=== Traitement pour 8664 images ===

=== Traitement pour la Taille 64x64 ===

=== Traitement du model DecisionTree ===

=== Traitement du model RandomForest ===

=== Traitement du model ExtraTrees ===

=== Traitement du model GradientBoosting ===

=== Traitement du model XGB ===

=== Traitement pour la Taille 128x128 ===

=== Traitement du model DecisionTree ===

=== Traitement du model RandomForest ===

=== Traitement du model ExtraTrees ===

=== Traitement du model GradientBoosting ===

=== Traitement du model XGB ===

=== Traitement pour la Taille 176x176 ===

=== Traitement du model DecisionTree ===

=== Traitement du model RandomForest ===

=== Traitement du model ExtraTrees ===

=== Traitement du model GradientBoosting ===

=== Traitement du model XGB ===

=== Traitement pour la Taille 224x224 ===

=== Traitement du model DecisionTree ===

=== Traitement du model RandomForest ===

=== Traitement du model ExtraTr

In [17]:
# === Résultats finaux sous forme de DataFrame ===
df_resultats = pd.read_csv("resultats_modeles_final.csv")
resumé=df_resultats.groupby("clf").agg({'score_train': 'mean', 'score_test': 'mean',"Temps(s)":'mean'})
display(resumé)

,score_train,score_test,Temps(s)
clf,,,
DecisionTree,1.000000,0.514604,22.1452
ExtraTrees,1.000000,0.695844,1.9712
GradientBoosting,0.915268,0.710836,1640.7248
KNN,0.747336,0.623692,0.0144
LR,0.993328,0.667204,3.8416
MLP,1.000000,0.743372,15.0468
RandomForest,1.000000,0.691880,3.6944
SVC,0.918276,0.735008,334.7444
XGB,1.000000,0.737008,90.8952


## 📊 Résultats finaux

Dans cette cellule, nous centralisons tous les résultats des expérimentations dans un **DataFrame final**.  

- Lecture du fichier `resultats_modeles_final.csv` contenant tous les scores obtenus pour chaque modèle, chaque taille d’image et chaque taille d’échantillon.  
- Calcul des **scores moyens** sur l’ensemble d’entraînement (`score_train`) et de test (`score_test`) pour chaque modèle.  
- Calcul du **temps moyen d’entraînement** (`Temps(s)`) pour chaque modèle afin d’avoir une idée de la performance en termes de rapidité.

> 💡 Cette étape permet de **résumer rapidement les performances globales** et le temps nécessaire pour chaque modèle avant de passer à l’analyse approfondie.  
> 
> La prochaine étape se fera dans le notebook **d’interprétation des modèles**, où nous analyserons les écarts entre train et test, identifierons les modèles sous- ou sur-appris, et déterminerons ceux qui sont les plus adaptés pour notre dataset, tout en prenant en compte leur temps d’exécution.
